# Classifying independent news&politics content creators and news organization channels

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys 
import matplotlib.ticker as ticker
import os
import glob
import plotly.graph_objects as go # install plotly if you dont have it
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_colwidth', None)
import re
import statistics

In [2]:
#remove parentheses and string between parentheses
def remove_nested_parens(input_str):
    """Returns a copy of 'input_str' with any parenthesized text removed. Nested parentheses are handled."""
    result = ''
    paren_level = 0
    for ch in input_str:
        if ch == '(':
            paren_level += 1
        elif (ch == ')') and paren_level:
            paren_level -= 1
        elif not paren_level:
            result += ch
    return result

In [3]:
# Organize Mutiple datasets that contain major news organization Youtube channels USA
df_news1 = pd.read_csv('../datasets/English news organisations/data_english_newsorgs.csv', engine ='python', sep ='\;')
df_news1['Publisher'] = df_news1['Publisher'].apply(remove_nested_parens)
df_news1 = df_news1['Publisher']

In [4]:
df_news2 = pd.read_csv('../datasets/English news organisations/news papers usa.csv', engine ='python', sep ='\;')
df_news2['Newspaper'] = df_news2['Newspaper'].apply(remove_nested_parens)
df_news2 = df_news2['Newspaper']

In [5]:
df_news3 = pd.read_csv('../datasets/English news organisations/indian media.csv', engine ='python', sep ='\;')
df_news3 = df_news3['Indian Media']

In [6]:
df_news4 = pd.read_csv('../datasets/English news organisations/usa media.csv', engine ='python', sep ='\;')
df_news4 = df_news4['Media channels']

In [7]:
df_news5 = pd.read_csv('../datasets/English news organisations/canadian media2.csv', engine ='python', sep ='\;')
df_news5 = df_news5['Canadian media']

In [8]:
df_news6 = pd.read_csv('../datasets/English news organisations/tabloid.csv', engine ='python', sep ='\;')
df_news6 = df_news6['Tabloid']

In [9]:
# make one list of mainstream news organizations
df_news = pd.DataFrame(df_news1.append([df_news2, df_news3, df_news4, df_news5, df_news6], 
                  ignore_index = True))
df_news.drop_duplicates()
df_news.rename(columns = {0:'mainstream news organizations'}, inplace = True)

<ipython-input-9-3ffc0bfd4959>:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_news = pd.DataFrame(df_news1.append([df_news2, df_news3, df_news4, df_news5, df_news6],


In [10]:
# make a copy to make the list upper case'
temp1 = df_news.copy()
temp1['mainstream news organizations'] = temp1['mainstream news organizations'].str.upper()
temp1['mainstream news organizations'] = temp1['mainstream news organizations'].str.replace(' ','')
temp1.head()

,mainstream news organizations
0,CNN
1,ABCNEWS
2,BBCNEWS
3,FOXNEWS
4,ALJAZEERA


In [11]:
# Channels data set
df_channels = pd.read_csv('../datasets/df_channels_en.tsv', compression="infer", sep="\t")
df_channels.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [12]:
# Keep channels that are in the category 'News & Politics'
df_filtered_channels = df_channels[( df_channels['category_cc']=='News & Politics')]

In [13]:
# Make a copy to make the column 'name_cc' upper case and remove white space
temp2 = df_filtered_channels.copy()
temp2['name_cc'] = temp2['name_cc'].str.upper()
temp2['name_cc'] = temp2['name_cc'].str.replace(' ','')

In [14]:
# Create a DataFrame that only contains major new organizations data

df_filtered_news_org = temp2[temp2['name_cc'].isin(temp1['mainstream news organizations'])]
df_filtered_news_org['creator_type'] = "news organization"
df_filtered_news_org.head()
len(df_filtered_news_org)

<ipython-input-14-915472aad19d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_news_org['creator_type'] = "news organization"


62

In [15]:
# Create a DataFrame that contains the rest(independent content creators)
df_filtered_news_ind = temp2[~temp2['name_cc'].isin(temp1['mainstream news organizations'])]
df_filtered_news_ind['creator_type'] = "independent"
len(df_filtered_news_ind)
df_filtered_news_ind.head()

<ipython-input-15-cc13cf2a1581>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_news_ind['creator_type'] = "independent"


,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,creator_type
212,News & Politics,2017-03-03,UCmphdqZNmqL72WJ2uyiNw5w,ABPNEWSHINDI,10800000,51298,340.0,2.087,independent
268,News & Politics,2015-03-23,UCx8Z14PpntdaxCt2hakbQLQ,THELALLANTOP,9120000,9423,438.0,2.087,independent
402,News & Politics,2009-08-08,UCj5RwDivLksanrNvkW0FB4w,GMAPUBLICAFFAIRS,6920000,20513,677.0,2.087,independent
510,News & Politics,2006-06-23,UCfwx98Wty7LhdlkxL5PZyLA,BARCROFTTV,7110000,2375,888.0,2.087,independent
555,News & Politics,2012-10-09,UC9k-yiEpRHMNVOnOi_aQK8w,INSIDEEDITION,6600000,18113,976.0,2.087,independent


In [16]:
frames = [df_filtered_news_org, df_filtered_news_ind]

In [17]:
df_filter_news_merge = pd.concat(frames)
df_filter_news_merge.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,creator_type
129,News & Politics,2006-08-26,UCttspZesZIDEwwpVIgoZtWQ,INDIATV,15177282,139814,199.0,2.087,news organization
133,News & Politics,2012-06-01,UCRWFSbif-RFENbBrSiez1DA,ABPNEWS,16274836,129027,207.0,2.087,news organization
337,News & Politics,2007-06-19,UCIvaYmXn910QMdemBG3v1pQ,ZEENEWS,9280000,102648,549.0,2.087,news organization
425,News & Politics,2005-10-02,UCupvZG-5ko_eiXAupbDfxWw,CNN,7500000,146299,720.0,2.087,news organization
527,News & Politics,2014-03-04,UCLXo7UDZvByw2ixzpQCufnA,VOX,6570000,1048,924.0,2.087,news organization


# Compare number of subscribers of independent creators and news organizations

In [18]:
# Compute median of subscribers and videos of top 30 channels from each category
df_filtered_news_org_30 = df_filtered_news_org.sort_values('subscribers_cc', ascending = False).head(30)
med_news_org = statistics.median(df_filtered_news_org_30['subscribers_cc'])
print(med_news_org)

1890000.0


In [19]:
df_filtered_news_ind_30 = df_filtered_news_ind.sort_values('subscribers_cc', ascending = False).head(30)
med_news_ind = statistics.median(df_filtered_news_ind_30['subscribers_cc'])
print(med_news_ind)

3975000.0


In [20]:
# What are the top 50 news channels in terms of subscribers? Weird that there are channels that don't produce english content
df_filter_news_merge.sort_values('subscribers_cc', ascending = False).head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,creator_type
133,News & Politics,2012-06-01,UCRWFSbif-RFENbBrSiez1DA,ABPNEWS,16274836,129027,207.0,2.087,news organization
129,News & Politics,2006-08-26,UCttspZesZIDEwwpVIgoZtWQ,INDIATV,15177282,139814,199.0,2.087,news organization
212,News & Politics,2017-03-03,UCmphdqZNmqL72WJ2uyiNw5w,ABPNEWSHINDI,10800000,51298,340.0,2.087,independent
337,News & Politics,2007-06-19,UCIvaYmXn910QMdemBG3v1pQ,ZEENEWS,9280000,102648,549.0,2.087,news organization
268,News & Politics,2015-03-23,UCx8Z14PpntdaxCt2hakbQLQ,THELALLANTOP,9120000,9423,438.0,2.087,independent


# Let's filter out channels that don't produce english content

In [21]:
# A function that detects characters that don't exist in english
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [22]:
isEnglish("dkssud12$")

True

In [23]:
path = '../datasets/metadata_chunks/'
file_list = os.listdir(path)

In [24]:
os.chdir(path)

In [25]:
#list all csv files only
print(os.getcwd())
csv_files = glob.glob('*.csv.gz')

/Users/sylena/ada-2022-project-chromegoldfish/datasets/metadata_chunks


In [26]:
# only reads specific columns since not all information is necessary.
# also only reads in subset of current dataframes since it is super massive
df = pd.DataFrame()
counter = 0

# you can add more categories if we want to do more analysis. category acts as a filter when reading in the chunks of the json files
category = ['News & Politics']
#append all files together
for file in csv_files:
            df_temp = pd.read_csv(file, usecols = ['categories','channel_id', 'description', 'title', 'upload_date', 'tags'])
            df_temp = df_temp[df_temp['categories'].isin(category)] # this will filter for only political channels now
            df = df.append(df_temp, ignore_index=True)
            counter += 1

            # temporary loop stopper. When we do our final analysis we will run it on all metadata chunks
            if counter == 1:
                break
df.head()

<ipython-input-26-68d5d8afa1ed>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp, ignore_index=True)


,categories,channel_id,description,tags,title,upload_date
0,News & Politics,UCpYohGnVR-Z4kjysac0W7jA,"CCTV does a story about online music lessons from http://ArtistWorks.com\nfeaturing a student of fingerstyle guitarist Martin Taylor, classical guitarist Jason Vieaux, and ArtistWorks founder David Butler.\n\nJust like Mike from nearby Oakland, you can learn from masters, Martin Taylor and Jason Vieaux.\n\nArtistWorks Online Music Schools work like this:\n1. Get a complete HD video library of lessons, plus tabs, notation and more\nSubmit practice videos and get personalized Video Exchange™ feedback from the legendary instructors\n2. Learn by watching the instructor's exchanges with other students\n3. Get inspiration from a worldwide community of enthusiastic players working together with the instructor\n\nThe ArtistWorks Music Campus includes instructions in jazz, acoustic, electric guitars, electric bass, bluegrass instruments, and several classical instruments. All the schools are instructed by famous musicians from major symphonies, such as The Philadelphia Orchestra.\n\nAn access to these teachers and to hundreds of HD lessons for only about a dollar a day. Your satisfaction is guaranteed. Find out more and join today!","china central television,china,cctv,online music schools,online education,music education,music education online,artistworks,News,music lessons,martin taylor,david butler",China Central Television (CCTV): Online Music Lessons,2012-05-04 00:00:00
1,News & Politics,UCpYHEAhSUx1hBn2SkvXvTrg,"The artist, producer and founding member of Beastie Boys Mike D unveils the ""grown up theme park"" he has curated at MOCA in Los Angeles.\nRead the full feature on NOWNESS: http://bit.ly/1h8ftVc\n\nWatch more art videos here: http://bit.ly/art-videos\n_______________________________________\n\nSubscribe to NOWNESS here: http://bit.ly/youtube-nowness\n\nLike NOWNESS on Facebook: http://bit.ly/facebook-nowness\nFollow NOWNESS on Twitter: http://bit.ly/twitter-nowness\nDaily exclusives for the culturally curious: http://bit.ly/nowness-com\nBehind the scenes on Instagram: http://bit.ly/instagram-nowness\nCurated stories on Tumblr: http://bit.ly/tumblr-nowness\nInspiration on Pinterest: http://bit.ly/pinterest-nowness\nStaff Picks on Vimeo: http://bit.ly/vimeo-nowness\nFollow NOWNESS on Google+: http://bit.ly/google-nowness","Art,Music,Culture,Video,Raf Simons,USA,Los Angeles,Beastie Boys,Tom Sachs,MOCA,Mike Mills,Geffen Contemporary,Transmission LA,Mike D,Cory Arcangel,Avant/Garde Diaries,Roy Choi","Mike D in ""Transmission LA: AV Club"" by Joe Termini",2012-04-23 00:00:00
2,News & Politics,UCpYFwrSRGSEmtv5SB0Ks-IA,"Get creative in the way you #RESIST\nThey don’t want you to realize your power.\nSo use it.\n\nBLEEDING AMERICA is an original makeup look designed by AUDFACED. Originally created in 2016 shortly after the election of the 45th President of the United States. I held onto this footage through much procrastination thinking of a better time to post it … President’s Day … Columbus Day / Indigenous People’s Day … 4th of July (twice) but as the big Midterm Elections of 2018 came I realized this was the time. Whether it’s voting season or not, I hope this piece can continually inspire and remind you that you are part of this world and you have been given unique gifts that you can use to do good in creative ways.\n\nMuch love and light.\n\nAUD xo\n\n#AUDFACED #BLEEDINGAMERICA\n\nMUSIC is “Nightmare”\nNEO FRANCE\nUsed With Permission\nhttps://soundcloud.com/neofrance/neo-nightmare\nThank You!\n\nPlease check out my political music video CRYING LIBERTY, a story that explores Lady Liberty in a demented America, exploring her sunken worlds searching to find her voice, regain her crown and remember who she is.\nProduced by AUDFACED + Risha Dee Films.\n\nWATCH “Crying Liberty” MUSIC VIDEO:\nhttp://www.youtube.com/watch?v=-yg027-X7BY\n\nFREE DOWNLOAD\n“Crying Liberty” SONG:\nhttps://soundcloud.com/audfaced/crying-liberty\n\nAUDF

In [27]:
# Removes duplicates of videos 
df_raw = df.drop_duplicates(subset=['title'])

In [28]:
# Get the channel id's and convert it to a dictionary to get names
channel_dict = dict(zip(df_channels.channel, df_channels.name_cc))

In [29]:
# Remap the channel_id with the channel names to improve who said what in our trend analysis
df = df_raw.replace({"channel_id": channel_dict})

In [30]:
df.head()

,categories,channel_id,description,tags,title,upload_date
0,News & Politics,ArtistWorks,"CCTV does a story about online music lessons from http://ArtistWorks.com\nfeaturing a student of fingerstyle guitarist Martin Taylor, classical guitarist Jason Vieaux, and ArtistWorks founder David Butler.\n\nJust like Mike from nearby Oakland, you can learn from masters, Martin Taylor and Jason Vieaux.\n\nArtistWorks Online Music Schools work like this:\n1. Get a complete HD video library of lessons, plus tabs, notation and more\nSubmit practice videos and get personalized Video Exchange™ feedback from the legendary instructors\n2. Learn by watching the instructor's exchanges with other students\n3. Get inspiration from a worldwide community of enthusiastic players working together with the instructor\n\nThe ArtistWorks Music Campus includes instructions in jazz, acoustic, electric guitars, electric bass, bluegrass instruments, and several classical instruments. All the schools are instructed by famous musicians from major symphonies, such as The Philadelphia Orchestra.\n\nAn access to these teachers and to hundreds of HD lessons for only about a dollar a day. Your satisfaction is guaranteed. Find out more and join today!","china central television,china,cctv,online music schools,online education,music education,music education online,artistworks,News,music lessons,martin taylor,david butler",China Central Television (CCTV): Online Music Lessons,2012-05-04 00:00:00
1,News & Politics,NOWNESS,"The artist, producer and founding member of Beastie Boys Mike D unveils the ""grown up theme park"" he has curated at MOCA in Los Angeles.\nRead the full feature on NOWNESS: http://bit.ly/1h8ftVc\n\nWatch more art videos here: http://bit.ly/art-videos\n_______________________________________\n\nSubscribe to NOWNESS here: http://bit.ly/youtube-nowness\n\nLike NOWNESS on Facebook: http://bit.ly/facebook-nowness\nFollow NOWNESS on Twitter: http://bit.ly/twitter-nowness\nDaily exclusives for the culturally curious: http://bit.ly/nowness-com\nBehind the scenes on Instagram: http://bit.ly/instagram-nowness\nCurated stories on Tumblr: http://bit.ly/tumblr-nowness\nInspiration on Pinterest: http://bit.ly/pinterest-nowness\nStaff Picks on Vimeo: http://bit.ly/vimeo-nowness\nFollow NOWNESS on Google+: http://bit.ly/google-nowness","Art,Music,Culture,Video,Raf Simons,USA,Los Angeles,Beastie Boys,Tom Sachs,MOCA,Mike Mills,Geffen Contemporary,Transmission LA,Mike D,Cory Arcangel,Avant/Garde Diaries,Roy Choi","Mike D in ""Transmission LA: AV Club"" by Joe Termini",2012-04-23 00:00:00
2,News & Politics,AUDFACED,"Get creative in the way you #RESIST\nThey don’t want you to realize your power.\nSo use it.\n\nBLEEDING AMERICA is an original makeup look designed by AUDFACED. Originally created in 2016 shortly after the election of the 45th President of the United States. I held onto this footage through much procrastination thinking of a better time to post it … President’s Day … Columbus Day / Indigenous People’s Day … 4th of July (twice) but as the big Midterm Elections of 2018 came I realized this was the time. Whether it’s voting season or not, I hope this piece can continually inspire and remind you that you are part of this world and you have been given unique gifts that you can use to do good in creative ways.\n\nMuch love and light.\n\nAUD xo\n\n#AUDFACED #BLEEDINGAMERICA\n\nMUSIC is “Nightmare”\nNEO FRANCE\nUsed With Permission\nhttps://soundcloud.com/neofrance/neo-nightmare\nThank You!\n\nPlease check out my political music video CRYING LIBERTY, a story that explores Lady Liberty in a demented America, exploring her sunken worlds searching to find her voice, regain her crown and remember who she is.\nProduced by AUDFACED + Risha Dee Films.\n\nWATCH “Crying Liberty” MUSIC VIDEO:\nhttp://www.youtube.com/watch?v=-yg027-X7BY\n\nFREE DOWNLOAD\n“Crying Liberty” SONG:\nhttps://soundcloud.com/audfaced/crying-liberty\n\nAUDFACED LINKS\n\nI really like updating daily on 

In [31]:
# Make a list of channels that contain non-english content
non_eng_channels = []

for index,row in df.iterrows():
    if (isEnglish(str(row['description'])) == False) and (row['channel_id'] not in non_eng_channels):
        non_eng_channels.append(row['channel_id'])


In [32]:
print(non_eng_channels)

['ArtistWorks', 'AUDFACED', 'Stop And Think', 'The Star Online', 'Prem Nepali', 'National Geographic', 'Uphill Media', 'Heroes of the Storm']


In [33]:
df_ = df[~df['channel_id'].isin(non_eng_channels)]

In [34]:
df_.head()

,categories,channel_id,description,tags,title,upload_date
1,News & Politics,NOWNESS,"The artist, producer and founding member of Beastie Boys Mike D unveils the ""grown up theme park"" he has curated at MOCA in Los Angeles.\nRead the full feature on NOWNESS: http://bit.ly/1h8ftVc\n\nWatch more art videos here: http://bit.ly/art-videos\n_______________________________________\n\nSubscribe to NOWNESS here: http://bit.ly/youtube-nowness\n\nLike NOWNESS on Facebook: http://bit.ly/facebook-nowness\nFollow NOWNESS on Twitter: http://bit.ly/twitter-nowness\nDaily exclusives for the culturally curious: http://bit.ly/nowness-com\nBehind the scenes on Instagram: http://bit.ly/instagram-nowness\nCurated stories on Tumblr: http://bit.ly/tumblr-nowness\nInspiration on Pinterest: http://bit.ly/pinterest-nowness\nStaff Picks on Vimeo: http://bit.ly/vimeo-nowness\nFollow NOWNESS on Google+: http://bit.ly/google-nowness","Art,Music,Culture,Video,Raf Simons,USA,Los Angeles,Beastie Boys,Tom Sachs,MOCA,Mike Mills,Geffen Contemporary,Transmission LA,Mike D,Cory Arcangel,Avant/Garde Diaries,Roy Choi","Mike D in ""Transmission LA: AV Club"" by Joe Termini",2012-04-23 00:00:00
3,News & Politics,SuperEliasTM,"THIS VIDEO WAS MADE A DAY AFTER THE KONY 2012 VIDEO WAS UPLOADED AND I DID NOT DO ANY RESEARCH ON IT. I AM TEARING UP IN THIS VIDEO FROM WHAT HAPPENED TO THOSE KIDS. I'm starting to have second thoughts about the video but I don't know for sure but I did not know that this did happened to many kids. So please stop with the hateful comments.\n\nI'm sorry I got some of my words mixed up.Before I filmed this video I watched the video again and put myself in the state of mind that I was one of those people trying to escape and seeing my brother that's trying to escape with me get murdered. I started tearing up and started to film. Here is the video you must see this...\nhttp://www.youtube.com/watch?v=Y4MnpzG5Sqc\nShare this video with your friends. They need to see this.\n\nThis video is to make Joseph Kony famous, not to celebrate him, but to raise support for his arrest and set a precedent for international justice.\nPledge your support at http://bit.ly/konydonate\n\nI'm happy I can help by making this video. Tell others about Kony on Facebook Twitter or on anything...","SuperEliasTM,SETM,Kony,2012,get,Joseph,famous,KONY,LRA,justice,Lord's,Resistance,Army,Invisible,Children,new,film,documentary,central,africa,child,soldiers",Stop Kony Make Him Famous!,2012-03-06 00:00:00
25332,News & Politics,ShadowRockZX,"Keybladeofvenom generously made a little AD for our forum. It's not the best, but it's just a little something for now. We'll make a better one at a later time. ^_^\nBy the way, if you do come and want to chat on the CB, well we have it set up to where guests can't talk on there until a Admin or Mod promotes them to member to prevent possible spam bots, so just wait for a Admin and PC them a request and we'll let you in.\nAre you a fan of the mini series Rise Of The Shadows? Well we got the forum for you! Come over to the ROTS (Rise Of The Shadows) Legacy Area (RLA for short)!\nWe got many interesting topics and discussions concerning the series. And you can even talk to the creators behind it! You can also suggest things you want to see in future episodes. But that's not all! We also have a nice little community where you can speak to others about your favorite MegaMan series or other video games or whatever! You may also post your own Patches, sprites, etc.\nWe need more people to liven the place up, so come over and chill with us.\nAnd if none of those reasons appeal to you..... You can come just to meet our forum clown: Javier! =D\nAdmins: ShadowRockZX, Keybladeofvenom, Licentio Vesico (Saberman)\nLink: http://s1.zetaboards.com/ROTS_Legacy_Area/index/\n\n\n\nFacebook:\nhttps://www.facebook.com/ShadowRockZX\nTwitter:\nhttps://twitter.com/ShadowRockX\nTwitch TV Channel:\nhttp://www.twitch.tv/shadowrockzx T-Shirts! Merchandise Store:\nhttp://sha